In [4]:
import os
import sys
import numpy as np
import pandas as pd
import tensorflow as tf

sys.path.append(os.path.abspath("../Model"))
import model_utils as mu

BASE_PATH = '../data_recordings'
RESULTS_DIR = '../Model/results'
os.makedirs(RESULTS_DIR, exist_ok=True)

MODEL_NAME = "Baseline_Adam"
MODEL_PATH = os.path.join(RESULTS_DIR, f"model_{MODEL_NAME}.keras")

print("TensorFlow:", tf.__version__)
print("Using SR:", mu.SR, "DURATION:", mu.DURATION, "TARGET_LENGTH:", mu.TARGET_LENGTH)


TensorFlow: 2.20.0
Using SR: 22050 DURATION: 3.0 TARGET_LENGTH: 66150


In [3]:
df = mu.load_manifest(BASE_PATH)
X, y, speakers = mu.load_and_process_data(df, BASE_PATH)

X_train, X_val, X_test, y_train, y_val, y_test, df_train, df_val, df_test = mu.split_dataset(X, y, speakers, df)

print("Train:", X_train.shape, "Val:", X_val.shape, "Test:", X_test.shape)


Loading 11225 audio files...
Successfully loaded: 11225, Failed: 0
Generating Mel-Spectrograms...
Train: (7859, 128, 130, 1) Val: (1681, 128, 130, 1) Test: (1685, 128, 130, 1)


In [ ]:
batch_size = 32
epochs = 15
steps_per_epoch = int(tf.math.ceil(len(X_train) / batch_size))

if os.path.exists(MODEL_PATH):
    print("Model already exists:", MODEL_PATH)
else:
    model = mu.build_model(
        input_shape=X_train.shape[1:],
        optimizer_name="adamw",
        dropout_rate=0.2,
        learning_rate=0.001,
        lr_schedule=None,
        weight_decay=1e-4,
        steps_per_epoch=steps_per_epoch,
        total_epochs=epochs,
        adapt_data=X_train
    )

    callbacks = [
        tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True, verbose=1),
        tf.keras.callbacks.CSVLogger(os.path.join(RESULTS_DIR, f"logs_{MODEL_NAME}.csv")),
        *mu.make_lr_callbacks(use_plateau=True),
    ]

    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=callbacks,
        verbose=1
    )

    loss, acc = model.evaluate(X_test, y_test, verbose=0)
    print("Test Accuracy:", acc, "Test Loss:", loss)

    model.save(MODEL_PATH)
    print("Saved:", MODEL_PATH)

    preds_prob = model.predict(X_test, verbose=0).flatten()
    pred_df = df_test.copy()
    pred_df["true_label"] = y_test
    pred_df["pred_prob"] = preds_prob
    pred_df["pred_label"] = (preds_prob > 0.5).astype(int)
    pred_df.to_csv(os.path.join(RESULTS_DIR, f"predictions_{MODEL_NAME}.csv"), index=False)


In [5]:
model = tf.keras.models.load_model(MODEL_PATH)
print("Loaded:", MODEL_PATH, "Input shape:", model.input_shape)

dummy = np.zeros((1, model.input_shape[1], model.input_shape[2], 1), dtype=np.float32)
_ = model.predict(dummy, verbose=0)
print("Warmup OK")


Loaded: ../Model/results\model_Baseline_Adam.keras Input shape: (None, 128, 130, 1)
Warmup OK


In [ ]:
import gradio as gr
import librosa

CAPTURE_SECONDS = 5
THRESHOLD = 0.5
SILENCE_TOP_DB = 35
MIN_NON_SILENT_SECONDS = 0.5

_, EXPECTED_MELS, EXPECTED_WIDTH, EXPECTED_CH = model.input_shape

def _mono_float32(y):
    y = np.asarray(y)
    if y.ndim == 2:
        y = y.mean(axis=1)
    y = y.astype(np.float32)
    peak = float(np.max(np.abs(y))) if y.size else 0.0
    if peak > 1.5:
        y = y / (peak + 1e-6)
    return y

def _non_silent_seconds(y, sr):
    intervals = librosa.effects.split(y, top_db=SILENCE_TOP_DB)
    if len(intervals) == 0:
        return 0.0
    return float(sum((e - s) for s, e in intervals) / sr)

def _make_model_input(y_proc, sr):
    mel = librosa.feature.melspectrogram(
        y=y_proc, sr=sr, n_mels=mu.N_MELS, fmax=mu.FMAX, hop_length=mu.HOP_LENGTH
    )
    mel_db = librosa.power_to_db(mel, ref=np.max).astype(np.float32)
    if mel_db.shape[1] < EXPECTED_WIDTH:
        mel_db = np.pad(mel_db, ((0,0),(0, EXPECTED_WIDTH - mel_db.shape[1])), mode="constant")
    elif mel_db.shape[1] > EXPECTED_WIDTH:
        mel_db = mel_db[:, :EXPECTED_WIDTH]
    return mel_db[np.newaxis, ..., np.newaxis]

def predict_ui(audio_input):
    if audio_input is None:
        return "No audio."

    sr, y = audio_input
    y = _mono_float32(y)

    if sr != mu.SR:
        y = librosa.resample(y, orig_sr=sr, target_sr=mu.SR)
        sr = mu.SR

    cap_n = int(CAPTURE_SECONDS * sr)
    if len(y) > cap_n:
        y = y[:cap_n]

    ns = _non_silent_seconds(y, sr)
    if ns < MIN_NON_SILENT_SECONDS:
        return f"Too silent ({ns:.2f}s non-silent)."

    y_proc = mu.preprocess_audio(y, sr=sr)
    X = _make_model_input(y_proc, sr)

    prob = float(model.predict(X, verbose=0)[0][0])
    pred = int(prob > THRESHOLD)

    status = "ACCEPTED" if pred == 1 else "REJECTED"
    conf = prob if pred == 1 else (1.0 - prob)

    return (
        f"{status}\n"
        f"P(class=1)={prob:.4f}\n"
        f"confidence={conf*100:.2f}%\n"
        f"non_silent={ns:.2f}s\n"
        f"segment_len={mu.DURATION:.1f}s"
    )

with gr.Blocks() as demo:
    gr.Markdown(f"# Speaker Verification\nRecord ~{CAPTURE_SECONDS}s, Predict.")
    audio = gr.Audio(type="numpy", sources=["microphone", "upload"])
    btn = gr.Button("Predict")
    out = gr.Textbox(lines=7)
    btn.click(predict_ui, inputs=audio, outputs=out)

demo.launch(show_error=True)

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "c:\Users\adamf\Desktop\student_debil\semestr7\WUT-IML-2025-2026\.venv\Lib\site-packages\uvicorn\protocols\http\h11_impl.py", line 410, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\adamf\Desktop\student_debil\semestr7\WUT-IML-2025-2026\.venv\Lib\site-packages\uvicorn\middleware\proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\adamf\Desktop\student_debil\semestr7\WUT-IML-2025-2026\.venv\Lib\site-packages\fastapi\applications.py", line 1135, in __call__
    await super().__call__(scope, receive, send)
  File "c:\Users\adamf\Desktop\student_debil\semestr7\WUT-IML-2025-2026\.venv\Lib\site-packages\starlette\applications.py", line 107, in __call__
    await self.middleware_stack(scope, receive, send)
  Fil